# Total Sales Pipeline
Raw layer from "Venda Total" pipeline

In [ ]:
from bs4 import BeautifulSoup
from io import BytesIO
from google.cloud import bigquery
import re
import os
import requests
import zipfile
import pandas as pd


In [ ]:
"""
Constants for the ANP Fuel Price Scraper
"""

BASE_URL = "https://www.gov.br/anp/pt-br/centrais-de-conteudo/paineis-dinamicos-da-anp/paineis-dinamicos-do-abastecimento/painel-dinamico-do-mercado-brasileiro-de-combustiveis-liquidos"

In [ ]:
"""
ANP Fuel Price Scraper
This script scrapes the ANP website for fuel prices and uploads the data to
Google Cloud Storage.
"""

response = requests.get(BASE_URL, verify=False)
if response.status_code == 200:
    page = BeautifulSoup(response.content, "html.parser")
else:
    raise Exception("Failed to retrieve data from the URL.")

pattern = re.compile(r"Clique\s+aqu.*Líquidos", re.IGNORECASE)
a_tag = page.find('a', string=pattern)
panel = None
if a_tag and 'href' in a_tag.attrs:
    panel_link = a_tag['href']
    panel_text = a_tag.get_text(strip=True)
    panel = {'text': panel_text, 'link': panel_link}
else:
    panel = None

data = None
data_link = page.find('a', string='Veja também a base de dados utilizada no painel')
if data_link and 'href' in data_link.attrs:
    link = data_link['href']
    text = data_link.get_text(strip=True)
    updated_data = data_link.next_sibling
    if updated_data:
        li_text = updated_data.get_text(strip=True).split("em ")[1][0:-1].strip()
    else:
        li_text = "Data não disponível"
    data = {'text': text, 'link': link, 'updated_date': li_text}
else:
    data = None

if data and data.get('link'):
    download_link = data.get('link')

    gcs_base_path = "general_sales"
    final_path = "/general_sales.zip"
    gcs_path = f"/{gcs_base_path}{final_path}"

    response = requests.get(download_link, verify=False)
    response.raise_for_status()
    zip_bytes = BytesIO(response.content)


In [ ]:
"""
Extracts the general sales data from a zip file in Google Cloud Storage,
"""

with zipfile.ZipFile(zip_bytes) as zf:
	for file_info in zf.infolist():
		with zf.open(file_info) as file:
			file_name = file_info.filename
			if file_info.filename == "Liquidos_Vendas_Atual.csv":
				df = pd.read_csv(file, sep=";", encoding="latin1")
				break


df = df.rename(columns={
	"Ano": "year",
	"Mês": "month",
	"Agente Regulado": "veto_txt_razao_social",
	"Código do Produto": "veto_txt_codigo_produto",
	"Nome do Produto": "veto_txt_nome_produto",
	"Descrição do Produto": "veto_txt_produto",
	"Região Origem": "veto_txt_origem_regiao",
	"UF Origem": "veto_txt_origem_estado",
	"Região Destinatário": "veto_txt_destino_regiao",
	"UF Destino": "veto_txt_destino_estado",
	"Mercado Destinatário": "veto_txt_mercado_destino",
	"Quantidade de Produto (mil m³)": "veto_qtd_volume_1000m3",

})

In [ ]:
"""
    this code removes unused years from the dataframe,
    filters the dataframe by start and end date,
    drops the year and product_code columns,
    converts the year, month, and day into a date column,
    get all datas to insert into BigQuery
    iterate over the dataframe and append the data to the corresponding month datafram
    create a hashmap to store the dataframes for each month
"""

df['veto_dat_venda'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=1)).astype('datetime64[us]')
df = df.drop(columns=['month', 'year'])
df['veto_dat_criacao'] = pd.Timestamp.now(tz='America/Sao_Paulo')

df['veto_qtd_volume_1000m3'] = df['veto_qtd_volume_1000m3'].astype(str).str.replace(',', '.').astype('float64')

df['veto_dat_venda'] = pd.to_datetime(df['veto_dat_venda']).dt.date

ordered_columns = [
    'veto_dat_venda',
    'veto_txt_razao_social',
    'veto_qtd_volume_1000m3',
    'veto_txt_produto',
    'veto_txt_origem_regiao',
    'veto_txt_origem_estado',
    'veto_txt_destino_regiao',
    'veto_txt_destino_estado',
    'veto_txt_mercado_destino',
    'veto_txt_codigo_produto',
    'veto_txt_nome_produto',
    'veto_dat_criacao'
]
df = df[ordered_columns]

In [ ]:
"""
Inserting data into BigQuery
"""

bq_client = bigquery.Client()
project_id = os.getenv("GOOGLE_CLOUD_PROJECT")
bq_dataset = "rw_ext_anp"
table_name = "venda_total"
table_id = f"{project_id}.{bq_dataset}.{table_name}"

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

try:
    job = bq_client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )
    job.result()
    print(f"  Data for {table_id} inserted successfully.")
except Exception as e:
    print(f"  Error inserting data for {table_id}: {str(e)}")

print("Data insertion completed!")